## 1.LeNet模型

In [11]:
import gluonbook as gb
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn
import time

In [12]:
net = nn.Sequential()
net.add(nn.Conv2D(channels = 6, kernel_size = 5, activation = 'sigmoid'),
       nn.MaxPool2D(pool_size = 2, strides = 2),
       nn.Conv2D(channels = 16, kernel_size = 5, activation  = 'sigmoid'),
       nn.MaxPool2D(pool_size = 2, strides = 2),
       # Dense会默认将（批量大小，通道，高，宽）形状的输入转换成
       # （批量大小， 通道 ＊ 高 ＊ 宽）形状的输入)
        nn.Dense(120, activation = 'sigmoid'),
        nn.Dense(84, activation = 'sigmoid'),
        nn.Dense(10))

In [13]:
'''
构造一个高和宽均为28的单通道数据样本，并逐层进行前向计算来查看每个层的输出形状
'''
X = nd.random.uniform(shape = (1, 1, 28, 28))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv3 output shape:	 (1, 6, 24, 24)
pool3 output shape:	 (1, 6, 12, 12)
conv4 output shape:	 (1, 16, 8, 8)
pool4 output shape:	 (1, 16, 4, 4)
dense3 output shape:	 (1, 120)
dense4 output shape:	 (1, 84)
dense5 output shape:	 (1, 10)


## 2.获取数据和训练

In [14]:
batch_size = 256
train_iter, test_iter = gb.load_data_fashion_mnist(batch_size = batch_size)

In [15]:
'''
尝试在gpu(0)上创建NDArray，如果成功则使用gpu(0)，否则仍然使用CPU
'''
def try_gpu4():
    try:
        ctx = mx.gpu()
        _ = nd.zeros((1, ), ctx = ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

ctx = try_gpu4()
ctx

cpu(0)

In [16]:
'''
as_in_context函数将数据复制到GPU上，例如gpu(0)
'''
def evaluate_accuracy(data_iter, net, ctx):
    acc = nd.array([0], ctx = ctx)
    for X, y in data_iter:
        # 如果ctx是GPU，将数据复制到GPU上
        X, y = X.as_in_context(ctx), y.as_in_context(ctx)
        acc += gb.accuracy(net(X), y)
    return acc.asscalar() / len(data_iter)

In [17]:
def train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs):
    print('training on', ctx)
    loss = gloss.SoftmaxCrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, start = 0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y)
            l.backward()
            trainer.step(batch_size)
            train_l_sum += l.mean().asscalar()
            train_acc_sum += gb.accuracy(y_hat, y)
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
             'time %.1f sec' % (epoch + 1, train_l_sum / len(train_iter),
                               train_acc_sum / len(train_iter),
                               test_acc, time.time() - start))

In [18]:
lr, num_epochs = 0.9, 5
net.initialize(force_reinit = True, ctx = ctx, init = init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on cpu(0)


AttributeError: module 'gluonbook' has no attribute 'accuracy'